# Hopfield Network

## Introduction

### Structure
![Structure](Hopfield-net.png)

### Updating
Here we update the network asynchronously by randomly choosing one node and update its ouput.
For a network with $N$ nodes:
$$s_{i}(t+1)=sgn(\sum_{j=1}^{N}w_{ij}s_{j}(t)-\theta_{i})$$
$$sgn(x)=1 (x\geq0)$$
$$sgn(x)=-1(otherwise)$$
$$W=W^{T}$$
$$W_{ii}>=0$$
Where $w_{ij}$ indicates the entry at $i_{th}$ row and $j_{th}$ column of $W$.
asynchronous
### Energy
#### Definition
$$E=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}w_{ij}s_{i}s_{j}+\sum_{i=1}^{N}\theta_{i}s_{i}$$
or in matrix form:
$$E=-\frac{1}{2}s^{T}Ws+\theta^{T}s$$
where:
$$s=\begin{bmatrix}s_{1}&s_{2}&\cdots&s_{N}\end{bmatrix}^{T}$$
$$\theta=\begin{bmatrix}\theta_{1}&\theta_{2}&\cdots&\theta_{N}\end{bmatrix}^{T}$$
#### Proof
* E is bounded
$$\lvert E\rvert\leq\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}\lvert w_{ij}\rvert \lvert s_{i}\rvert\lvert s_{j}\rvert+\sum_{i=1}^{N}\lvert\theta_{i}\rvert\lvert s_{i}\rvert$$
* E is monotonically decreasing

Assume that the $k_{th}$ node has been updated:
$$s(t+1)=s(t)+\Delta s(t)$$
$$\Delta s(t)=\epsilon_{k}\Delta s_{k}(t)$$
\begin{equation}
\begin{split}
\Delta E&=E(t+1)-E(t)\\
&=-\frac{1}{2}\left(\left(s(t)+\Delta s(t)\right)^{T}W\left(s(t)+\Delta s(t)\right)\right)+\theta^{T}\left(s(t)+\Delta s(t)\right)\\
&-\left(-\frac{1}{2}s(t)^{T}Ws(t)+\theta^{T}s(t)\right)\\
&=-\frac{1}{2}\left(\Delta s(t)^{T}Ws(t)+ s(t)^{T}W\Delta s(t)+\Delta s(t)^{T}W\Delta s(t)\right)+\theta^{T}\Delta s(t)\\
&=-\frac{1}{2}\left[\Delta s_{k}(t)\sum_{j=1}^{N}w_{kj}s_{j}(t)+\Delta s_{k}(t)\sum_{i=1}^{N}w_{ik}s_{i}(t)+w_{kk}\Delta s_{k}^{2}(t)\right]+\Delta s_{k}(t)\theta_{k}
\end{split}
\end{equation}
where $\epsilon_{k}$ denotes the vector with $k_{th}$ entry equal to 1.
Given that $w_{ij}=w_{ji}$,
$$\Delta E=-\Delta s_{k}(t)\left(\sum_{j=1}^{N}w_{kj}s_{j}(t)-\theta_{k}\right)-\frac{1}{2}w_{kk}\Delta s_{k}^{2}(t)
$$
Note that$\Delta s_{k}(t)\left(\sum_{j=1}^{N}w_{kj}s_{j}(t)-\theta_{k}\right)\geq0$ and that $w_{kk}\geq0$, we obtain:
$$\Delta E\leq-\frac{1}{2}w_{kk}\Delta s_{k}^{2}(t)\leq 0$$
Hence, E is monotonically decreasing. 

Thus, we have the conclusion that the energy of the network will converge to a local minimum after several iterations. 

### A numerical example for convergence

In [1]:
import numpy as np
import random

In [2]:
def updateState(state,W,theta,verbose=False,reshape=None):
    networkSize=np.shape(state)[0]
    if np.shape(state)!=np.shape(theta):
        raise(ValueError("The shapes of states and thresholds didn't match."))
    if networkSize!=np.shape(W)[0]:
        raise(ValueError("The shapes of states and W didn't match."))
    if np.shape(W)[0]!=np.shape(W)[1]:
        raise(ValueError("W must be a square matrix."))
    lastState=None
    iteration=0
    n=0
    k=random.randint(0,networkSize-1)
    if verbose:
        if reshape==None:
            print("Original state:\n{}".format(state))
        else:
            print("Original state:\n{}".format(np.reshape(state,reshape)))
    while n<networkSize:
        iteration+=1
        #lastState=np.array(state,copy=True)
        lastState=np.copy(state)
        if np.sum(W[k,:]*state)-theta[k]>0:
            state[k]=1
        elif np.sum(W[k,:]*state)-theta[k]<0:
            state[k]=-1
        else:
            state[k]=0
        if type(lastState)==type(None) or np.any(lastState!=state):
            n=0
        else:
            n+=1
        if verbose:
            print("Node number {} has been updated at epoch {}.".format(k,iteration))
            if reshape==None:
                print("New state:\n{}".format(state))
            else:
                print("New state:\n{}".format(np.reshape(state,reshape)))
        if k<networkSize-1:
            k+=1
        else:
            k=0
    return state

In [3]:
initState=np.array([1,-1,-1,-1])
W=np.array([[0,2,2,2],[2,0,2,2],[2,2,0,2],[2,2,2,0]])
theta=np.array([1,2,3,4])
updateState(initState,W,theta,verbose=True)

Original state:
[ 1 -1 -1 -1]
Node number 0 has been updated at epoch 1.
New state:
[-1 -1 -1 -1]
Node number 1 has been updated at epoch 2.
New state:
[-1 -1 -1 -1]
Node number 2 has been updated at epoch 3.
New state:
[-1 -1 -1 -1]
Node number 3 has been updated at epoch 4.
New state:
[-1 -1 -1 -1]
Node number 0 has been updated at epoch 5.
New state:
[-1 -1 -1 -1]


array([-1, -1, -1, -1])

In [4]:
initState=np.array([1,1,1,-1])
updateState(initState,W,theta,verbose=True,reshape=None)

Original state:
[ 1  1  1 -1]
Node number 2 has been updated at epoch 1.
New state:
[ 1  1 -1 -1]
Node number 3 has been updated at epoch 2.
New state:
[ 1  1 -1 -1]
Node number 0 has been updated at epoch 3.
New state:
[-1  1 -1 -1]
Node number 1 has been updated at epoch 4.
New state:
[-1 -1 -1 -1]
Node number 2 has been updated at epoch 5.
New state:
[-1 -1 -1 -1]
Node number 3 has been updated at epoch 6.
New state:
[-1 -1 -1 -1]
Node number 0 has been updated at epoch 7.
New state:
[-1 -1 -1 -1]
Node number 1 has been updated at epoch 8.
New state:
[-1 -1 -1 -1]


array([-1, -1, -1, -1])

### Design $W$
Design $W$ to store $M$ samples in $X=\begin{bmatrix}X_{1}&X_{2}&\cdots&X_{M}\end{bmatrix}$
#### Hebb's learning rule
$$w_{ij}=\alpha\sum_{k=1}^{M}x_{i}^{k}x_{j}^{k},(i\neq j)$$
$$w_{ii}=0$$
or in matrix form
$$W=\alpha\sum_{k=1}^{M}\left(X_{k}X_{k}^{T}-I\right)$$
Assume we need to store two samples: H and V. 


In [5]:
cross=[
-1,-1,+1,-1,-1,
-1,-1,+1,-1,-1,
+1,+1,+1,+1,+1,
-1,-1,+1,-1,-1,
-1,-1,+1,-1,-1]#Cross
diamond=[
-1,-1,+1,-1,-1,
-1,+1,-1,+1,-1,
+1,-1,-1,-1,+1,
-1,+1,-1,+1,-1,
-1,-1,+1,-1,-1]#Diamond
X=np.array([cross,diamond]).T
alpha=0.5
dim,size=np.shape(X)
W=np.zeros([dim,dim],dtype=np.float)

for i in range(size):
    x=np.reshape(X[:,i],[dim,1])
    W=W+x*x.T
W=W-size*np.eye(dim)
W=alpha*W
initState=-np.ones(dim)
theta=np.ones(dim)
updateState(initState,W,theta,verbose=True,reshape=[5,5])

Original state:
[[-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 6 has been updated at epoch 1.
New state:
[[-1. -1. -1. -1. -1.]
 [-1.  1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 7 has been updated at epoch 2.
New state:
[[-1. -1. -1. -1. -1.]
 [-1.  1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 8 has been updated at epoch 3.
New state:
[[-1. -1. -1. -1. -1.]
 [-1.  1. -1.  1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 9 has been updated at epoch 4.
New state:
[[-1. -1. -1. -1. -1.]
 [-1.  1. -1.  1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 10 has been updated at epoch 5.
New state:
[[-1. -1. -1. -1. -1.]
 [-1.  1. -1.  1. -1.]
 [ 1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
Node number 11 ha

array([-1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.])

It's interesting that if we use a new initial state we will get a the inversion of the diamond in the training sample. 

In [6]:
initState=np.ones(dim)
theta=np.ones(dim)
finalState=updateState(initState,W,theta,reshape=[5,5])
print("Final state:\n{}".format(np.reshape(finalState,[5,5])))

Final state:
[[ 1.  1. -1.  1.  1.]
 [ 1. -1.  1. -1.  1.]
 [-1.  1.  1.  1. -1.]
 [ 1. -1.  1. -1.  1.]
 [ 1.  1. -1.  1.  1.]]


#### Pseudo Inverse
To be continued...

### Hopfield network and TSP
The [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem) asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?" It is an NP-hard problem in combinatorial optimization, important in operations research and theoretical computer science.

TSP is a special case of the travelling purchaser problem and the vehicle routing problem. 
#### Design energy function
Suppose we have $N$ cities in total. We represent the route of the salesman with a matrix $V_{N\times N}$. $v_{ij}\in \{0,1\}$ where $v_{ij}$ denotes the element in the $i_{th}$ row and the $j_{th}$ column of $V$. $v_{ij}=1$ indicates that the $i_{th}$ city is reached at step $j$. We use matrix $D$ to denote the distances among cities where the element in the $i_{th}$ row and $j_{th}$ column $d_{ij}$ denotes the cost to go from city $i$ to city $j$ (e.g. distance between two cities).

Now we define an energy function as follow:
$$E=\frac{A}{2}E_{1}+\frac{B}{2}E_{2}+\frac{C}{2}E_{3}+\frac{D}{2}E_{4}$$
where,
$$E_{1}=\sum_{k=1}^{N}\sum_{i,j,i\neq j}v_{ki}v_{kj}$$
$$E_{2}=\sum_{k=1}^{N}\sum_{i,j,i\neq j}v_{ik}v_{jk}$$
$$E_{3}=\left(N-\sum_{i=1}^{N}\sum_{j=1}^{N}v_{ij}\right)^{2}$$
$$E_{4}=\sum_{k=1}^{N}\sum_{i=1}^{N}\sum_{j=1}^{N}d_{ij}v_{ik}(v_{j,(k-1)}+v_{j,(k+1)})$$

$E_{1}$ and $E_{2}$ penalize the possible violations over the constraints that each row/column of the matrix $V$ contains at most one 1, since each city should be vistied once and the salesman can visit one city at a time.

$E_{3}$ indicates the constrain that a total of $N$ cities must be visited. 

$E_{4}$ represents the cost we want to minimize.